In [1]:
import tensorflow as tf
import numpy as np
import random, json, string, pickle
import keras
import keras.layers
import keras.models
import keras.optimizers
from keras.layers.wrappers import TimeDistributed
import keras.layers.embeddings
import keras.preprocessing.text
import keras.preprocessing.sequence
import keras.callbacks


Using TensorFlow backend.


In [2]:
[ids, titles, labels, food2id, id2food, class_weights, class_weights2] = pickle.load( open( "ids_labels_nomissing_500_titles.p", "rb" ) )

IOError: [Errno 2] No such file or directory: 'ids_labels_nomissing_500_titles.p'

In [ ]:
vocabularySize = 1000 
# Split Title into words, and define a vocabulary with the most common words.
tokenizer = keras.preprocessing.text.Tokenizer(nb_words = vocabularySize, \
                                               filters = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~\t\n')
#Fit on titleStrings 
tokenizer.fit_on_texts(titleStrings)

# Convert the title names into sequences of word ids using our vocabulary.
titleSequences = tokenizer.texts_to_sequences(titleStrings)

# Keep dictionaries that map ids -> words, and words -> ids.
word2id = tokenizer.word_index
id2word = {idx: word for (word, idx) in word2id.items()}
# Find the sentence with most words.
maxTitleLength = max([len(seq) for seq in titleSequences]) 

#pad sequences
data = keras.preprocessing.sequence.pad_sequences(titleSequences, 
           maxlen = (maxTitleLength + 1), padding = 'post', truncating = 'post')

id2word[0] = 'END'
word2id['END'] = 0

In [ ]:
print(data.shape)  
print(string.join([id2word[idx] for idx in data[0]], " "))

In [3]:
#build model...

#Define Shape of input batchSize
#output of Previous model X maxTitleLength
recipes_title = keras.layers.Input(batch_shape=(4096 + 500 , maxTitleLength), name = "input")
#recipes_title = keras.layers.Input(batch_shape=(4096 + 500), name = "input")


# Build a matrix of size vocabularySize x 300 where each row corresponds to a "word embedding" vector.
# This layer will convert replace each word-id with a word-vector of size 300.
embeddings = keras.layers.embeddings.Embedding(vocabularySize, 300, name = "embeddings")(recipes_title)

# Pass the word-vectors to the LSTM layer.
# We are setting the hidden-state size to 512.
hiddenStates = keras.layers.SimpleRNN(512, return_sequences = True, input_shape=(maxTitleLength, 300), name = "rnn")(embeddings)

# Apply a linear (Dense) layer of size 512 x 256 to the outputs of the LSTM at each time step.
denseOutput = TimeDistributed(keras.layers.Dense(vocabularySize), name = "linear")(hiddenStates)
predictions = TimeDistributed(keras.layers.Activation("softmax"), name = "softmax")(denseOutput)

# Build the computational graph by specifying the input, and output of the network.
model = keras.models.Model(input = recipes_title, output = predictions)

#For Recurrent Neural Networks, a type of
# optimization called RMSprop is preferred instead of the standard SGD udpates.
model.compile(loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.RMSprop(lr = 0.001))

print(model.summary())


NameError: name 'maxTitleLength' is not defined

In [ ]:
#Train Model

inputData = data[:, :-1]  # words 1, 2, 3, ... , (n-1)
outputData = data[:, 1:]  # words 2, 3, 4, ... , (n)

outputLabels = np.expand_dims(outputData, -1)


checkpointer = keras.callbacks.ModelCheckpoint(filepath="my_weights_RNN.hdf5", save_weights_only = True, \
                                               save_best_only = True, monitor = 'loss')
model.fit(inputData, outputLabels, batch_size = 256, nb_epoch = 10, callbacks = [checkpointer])

model.save_weights('my_recipe_model.hdf5')


In [ ]:
#Build Inference model

